## Make Detection with the Trained Model

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd

import pickle

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

objc[95972]: Class CaptureDelegate is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x106708860) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x2856f6480). One of the two will be used. Which one is undefined.
objc[95972]: Class CVWindow is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x105400a68) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x2856f64d0). One of the two will be used. Which one is undefined.
objc[95972]: Class CVView is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x105400a90) and /Users/fuixlabsdev1/Programming/PP/graduation-th

### Reconstruct the input structure

In [2]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

### Setup some important functions

In [3]:
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in IMPORTANT_LMS:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [15]:
VIDEO_PATH = "../data/plank/plank_test.mov"
VIDEO_PATH2 = "../data/plank/bad/plank_bad_3.mp4"
VIDEO_PATH3 = "../data/plank/bad/plank_bad_high_4.mp4"

### Make detection with Scikit learn model

In [4]:
# Load model
with open("./plank_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [17]:
cap = cv2.VideoCapture(VIDEO_PATH3)
current_stage = ""
prediction_probability_threshold = 0.65

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        # Reduce size of a frame
        image = rescale_frame(image, 60)

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        if not results.pose_landmarks:
            print("No human found")
            continue

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=2), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))

        # Make detection
        try:
            # Extract keypoints from frame for the input
            row = extract_important_keypoints(results)
            X = pd.DataFrame([row], columns=HEADERS[1:])

            # Make prediction and its probability
            predicted_class = trained_model.predict(X)[0]
            prediction_probability = trained_model.predict_proba(X)[0]
            # print(predicted_class, prediction_probability)

            # Evaluate model prediction
            if predicted_class == "C" and prediction_probability[prediction_probability.argmax()] >= prediction_probability_threshold:
                current_stage = "Correct"
            elif predicted_class == "L" and prediction_probability[prediction_probability.argmax()] >= prediction_probability_threshold: 
                current_stage = "Low back"
            elif predicted_class == "H" and prediction_probability[prediction_probability.argmax()] >= prediction_probability_threshold: 
                current_stage = "High back"
            else:
                current_stage = "UNKNOWN"
            
            # Visualization
            # Status box
            cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

            # Display class
            cv2.putText(image, "CLASS", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, current_stage, (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display probability
            cv2.putText(image, "PROB", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(prediction_probability[np.argmax(prediction_probability)], 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")
        
        cv2.imshow("CV2", image)
        
        # Press Q to close cv2 window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1, 5):
        cv2.waitKey(1)
  

No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found
No human found


### Make detection with Deep Learning Model

In [9]:
# Load model
with open("./plank_model_deep_learning.pkl", "rb") as f:
    deep_learning_model = pickle.load(f)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-10-06 21:30:30.067696: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-06 21:30:30.067792: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
cap = cv2.VideoCapture(VIDEO_PATH3)
current_stage = ""
prediction_probability_threshold = 0.7

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        # Reduce size of a frame
        image = rescale_frame(image, 50)

        # Recolor image from BGR to RGB for mediapipe
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        if not results.pose_landmarks:
            print("No human found")
            continue

        # Recolor image from BGR to RGB for mediapipe
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=2), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=1))

        # Make detection
        try:
            # Extract keypoints from frame for the input
            row = extract_important_keypoints(results)
            X = pd.DataFrame([row], columns=HEADERS[1:])

            # Make prediction and its probability
            prediction = deep_learning_model.predict(X)
            predicted_class = np.argmax(prediction, axis=1)[0]

            prediction_probability = max(prediction.tolist()[0])
            print(row)

            # Evaluate model prediction
            if predicted_class == 0 and prediction_probability >= prediction_probability_threshold:
                current_stage = "Correct"
            elif predicted_class == 1 and prediction_probability >= prediction_probability_threshold: 
                current_stage = "Low back"
            elif predicted_class == 2 and prediction_probability >= prediction_probability_threshold: 
                current_stage = "High back"
            else:
                current_stage = "Unknown"

            # Visualization
            # Status box
            cv2.rectangle(image, (0, 0), (550, 60), (245, 117, 16), -1)

            # # Display class
            cv2.putText(image, "DETECTION", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, current_stage, (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # # Display class
            cv2.putText(image, "CLASS", (350, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(predicted_class), (345, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # # Display probability
            cv2.putText(image, "PROB", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(prediction_probability, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")
        
        cv2.imshow("CV2", image)
        
        # Press Q to close cv2 window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
    for i in range (1, 5):
        cv2.waitKey(1)
  

No human found
No human found
No human found
1/1 [==============================] - 0s 12ms/step
[0.6020539402961731, 0.350770503282547, 0.18618133664131165, 0.22720938920974731, 0.6266858577728271, 0.3597281575202942, 0.024433838203549385, 0.976525604724884, 0.571766197681427, 0.37409594655036926, 0.01826258935034275, 0.9847692251205444, 0.5949684381484985, 0.38205498456954956, 0.1278994232416153, 0.14898861944675446, 0.503703236579895, 0.3805868625640869, 0.1399257779121399, 0.07331332564353943, 0.5666775107383728, 0.45738935470581055, 0.22621282935142517, 0.019617794081568718, 0.4582778811454773, 0.34603771567344666, 0.29719895124435425, 0.009459441527724266, 0.6157313585281372, 0.22217634320259094, -0.015413573011755943, 0.9994311928749084, 0.5961870551109314, 0.23548442125320435, 0.014728080481290817, 0.9992488026618958, 0.6050621271133423, 0.26277193427085876, 0.313665509223938, 0.31883949041366577, 0.6101233959197998, 0.26025089621543884, 0.35176849365234375, 0.06464847177267075

: 

In [ ]:
prediction_probability